Is it possible to do a SB and SDB, WOSB, VOSB analysis on all BIL, IRA spending at federal level?

In [1]:
import pandas as pd
import duckdb
from pathlib import Path
from zipfile import ZipFile

pd.options.display.float_format = '{:,.0f}'.format
pd.options.display.max_rows = None

In [2]:
%load_ext sql
%sql duckdb:///:default:

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

duckdb.sql('PRAGMA disable_progress_bar;')

There's a new jupysql version available (0.10.5), you're running 0.10.3. To upgrade: pip install jupysql --upgrade


Connecting to 'duckdb:///:default:'

In [3]:
#constants and paths
filepath = 'Contracts_PrimeTransactions_2023-12-01_H14M22S19_1.csv'

In [ ]:
%%sql
DESCRIBE
SELECT
  *
FROM
  read_csv_auto ('{{filepath}}')
LIMIT
  10

In [5]:
cols = [
#size
'contracting_officers_determination_of_business_size_code',

#SDB
'self_certified_small_disadvantaged_business',
'small_disadvantaged_business',
'c8a_program_participant',

#WOSB
'woman_owned_business',
'women_owned_small_business',
'economically_disadvantaged_women_owned_small_business',
'joint_venture_women_owned_small_business',
'joint_venture_economic_disadvantaged_women_owned_small_bus',
    
#VOSB
'veteran_owned_business',
'service_disabled_veteran_owned_business',

#dollars
'federal_action_obligation',

#IIJA info
'disaster_emergency_fund_codes_for_overall_award',
    
#disaggregated
'minority_owned_business'
'subcontinent_asian_asian_indian_american_owned'
'asian_pacific_american_owned_business'
'black_american_owned_business'
'hispanic_american_owned_business'
'native_american_owned_business'
'other_minority_owned_business'
]

In [6]:
%%sql
BIL_outlays =<<
SELECT
  CASE
    WHEN black_american_owned_business = 't' THEN 'Black_Am_Owned'
    WHEN hispanic_american_owned_business = 't' THEN 'Hispanic_Am_Owned'
    WHEN asian_pacific_american_owned_business = 't' THEN 'Asian_Pac_Am_Owned'
    WHEN subcontinent_asian_asian_indian_american_owned_business = 't' THEN 'South_Asian_Am_Owned'
    WHEN native_american_owned_business = 't' THEN 'Native_Am_Owned'
    WHEN other_minority_owned_business = 't' THEN 'Other_Minority_Owned'
    ELSE 'None'
  END AS Minority_Representation,
  SUM(federal_action_obligation) AS total,
  SUM(
    IF (
      contracting_officers_determination_of_business_size_code = 'S',
      federal_action_obligation,
      0
    )
  ) AS small_business,
  SUM(
    IF (
      contracting_officers_determination_of_business_size_code = 'S'
      AND (
        self_certified_small_disadvantaged_business = 't'
        OR small_disadvantaged_business = 't'
        OR c8a_program_participant = 't'
      ),
      federal_action_obligation,
      0
    )
  ) AS SDB,
  SUM(
    IF (
      contracting_officers_determination_of_business_size_code = 'S'
      AND (
        woman_owned_business = 't'
        OR women_owned_small_business = 't'
        OR economically_disadvantaged_women_owned_small_business = 't'
        OR joint_venture_women_owned_small_business = 't'
        OR joint_venture_economic_disadvantaged_women_owned_small_bus = 't'
      ),
      federal_action_obligation,
      0
    )
  ) AS WOSB,
  SUM(
    IF (
      contracting_officers_determination_of_business_size_code = 'S'
      AND (
        veteran_owned_business = 't'
        OR service_disabled_veteran_owned_business = 't'
      ),
      federal_action_obligation,
      0
    )
  ) AS VOSB 
FROM
  read_csv_auto ('{{filepath}}')
WHERE
  disaster_emergency_fund_codes_for_overall_award LIKE '1%'
  OR disaster_emergency_fund_codes_for_overall_award LIKE 'Z%'
GROUP BY CUBE
  (Minority_Representation)
ORDER BY
  total desc

,Minority_Representation,total,small_business,SDB,WOSB,VOSB
0,NaN,"64,146,985,335","5,811,362,474","2,956,568,462","1,028,874,567","515,369,097"
1,None,"61,769,560,995","3,562,108,616","913,439,851","448,298,811","324,335,962"
2,Native_Am_Owned,"854,075,252","849,693,498","825,809,457","55,100,071","19,714,715"
3,Hispanic_Am_Owned,"444,303,486","438,701,687","377,676,745","116,592,470","44,320,804"
4,South_Asian_Am_Owned,"432,217,507","346,542,684","321,944,957","145,284,551","6,247,774"
5,Asian_Pac_Am_Owned,"336,881,261","327,245,303","242,053,652","157,151,777","13,371,359"
6,Black_Am_Owned,"284,856,012","267,958,430","257,524,851","97,140,709","106,561,974"
7,Other_Minority_Owned,"25,090,824","19,112,256","18,118,950","9,306,179","816,509"


In [7]:
print(BIL_outlays.round(0).fillna('TOTAL').set_index('Minority_Representation')\
      .pipe(lambda _df:_df.rename({x:x.replace('_'," ") for x in _df.index}))     
     )

                                 total  small_business           SDB  \
Minority_Representation                                                
TOTAL                   64,146,985,335   5,811,362,474 2,956,568,462   
None                    61,769,560,995   3,562,108,616   913,439,851   
Native Am Owned            854,075,252     849,693,498   825,809,457   
Hispanic Am Owned          444,303,486     438,701,687   377,676,745   
South Asian Am Owned       432,217,507     346,542,684   321,944,957   
Asian Pac Am Owned         336,881,261     327,245,303   242,053,652   
Black Am Owned             284,856,012     267,958,430   257,524,851   
Other Minority Owned        25,090,824      19,112,256    18,118,950   

                                 WOSB        VOSB  
Minority_Representation                            
TOTAL                   1,028,874,567 515,369,097  
None                      448,298,811 324,335,962  
Native Am Owned            55,100,071  19,714,715  
Hispanic Am Owned  

In [8]:
with pd.option_context ('display.float_format', '{:,.2%}'.format):
    print(BIL_outlays.set_index('Minority_Representation')\
        .iloc[0]\
        .pipe(lambda srs:srs.div(srs.total))
         )

total            100.00%
small_business     9.06%
SDB                4.61%
WOSB               1.60%
VOSB               0.80%
Name: nan, dtype: float64


In [9]:
with pd.option_context ('display.float_format', '{:,.2%}'.format):
    print(BIL_outlays.fillna('TOTAL')\
          .set_index('Minority_Representation')\
          .pipe(lambda _df:_df.div(_df.loc['TOTAL']))\
          .pipe(lambda _df:_df.rename({x:x.replace('_'," ") for x in _df.index}))     
         )

                          total  small_business     SDB    WOSB    VOSB
Minority_Representation                                                
TOTAL                   100.00%         100.00% 100.00% 100.00% 100.00%
None                     96.29%          61.30%  30.90%  43.57%  62.93%
Native Am Owned           1.33%          14.62%  27.93%   5.36%   3.83%
Hispanic Am Owned         0.69%           7.55%  12.77%  11.33%   8.60%
South Asian Am Owned      0.67%           5.96%  10.89%  14.12%   1.21%
Asian Pac Am Owned        0.53%           5.63%   8.19%  15.27%   2.59%
Black Am Owned            0.44%           4.61%   8.71%   9.44%  20.68%
Other Minority Owned      0.04%           0.33%   0.61%   0.90%   0.16%


In [10]:
unicor_uei = ['Q9M6NBKH74G3',
 'SYLDKUFLV1H4',
 'DX5EKNZFA3N5',
 'DJNLDAUMKEJ5',
 'QSVDL5JLEEJ6',
 'JAMWLNJ9J5P6',
 'C8CLXN6LVNF8',
 'MH4CB9DKTNQ4',
 'W66EAQMDW725',
 'RY1UXHB1M1K5',
 'E8NTQR3CJEF8',
 'K1DNSJ1RT9A1',
 'DRDXM1XD2KK8',
 'TZYMUE5NK8M4',
 'T422H2MGNEP6',
 'KHFLCLB4BW91',
 'JQLDEL8SSF24',
 'NL9XTCND2B61',
 'S1NSFTYGMES9',
 'VSNRFJTTPMX6',
 'T1KSAHQRAAM7',
 'JKZXA7K7CJG2',
 'P95UX39ABTK7',
 'V4W9UULK21B7',
 'LKJLV1UHQNL8',
 'S118NFU2A4H3',
 'NS4GGWZGXJG8',
 'GCXKXNG653F9',
 'LHMZK6ZF6S87',
 'C8BXEEATNDL1',
 'HS2LL6F188B8',
 'Y6DKA9CH9HG8',
 'JG5ELMLTFKW3',
 'N1JBKE5TJ1J5',
 'EVKUQPQXME98',
 'XKK5YFSFYSS2',
 'CH2KGNMYKBZ4',
 'MH2LK48RW8K9',
 'C4KAFNUZUNJ9',
 'EB6TWP1VMZB5',
 'D5ENM348KWL9',
 'RLUNF7CEP6U6',
 'L2MHVK5MLVE3',
 'Z4QSJZHMQZ28',
 'MV2UKGNJRU25',
 'J4EEUF33BA75',
 'LMMMUEPESAL1',
 'QAW8DCW831M6',
 'HHXDFJHVQE43',
 'Z21JKYZ3S5B1',
 'UPCNMFE7BMM6',
 'LS1WZ2N7J8L4',
 'T2JVV8B631L3',
 'WQ25KCDAFWT8',
 'ZFVQNVDJHML4',
 'GPVSZFFUERL7',
 'G2FGNLLUJNL2',
 'GW4TFWJJF5Y9',
 'GFACXJC42P43',
 'R9B8T8PFG3K9',
 'GVJQZ583KMF7',
 'MVQ1B9LLX4V9']

Ed_PIID = ['EDFSA11D0012',
 'EDFSA12D0003',
 'EDFSA12D0005',
 'EDFSA12D0006',
 'EDFSA12D0007',
 'EDFSA12D0012',
 'EDFSA13D0001']

excluded_countries = ['AFGHANISTAN',
 'ALBANIA',
 'ARMENIA',
 'AUSTRALIA',
 'AUSTRIA',
 'AZERBAIJAN',
 'BELARUS',
 'BELGIUM',
 'BELIZE',
 'BENIN',
 'BOSNIA AND HERZEGOVINA',
 'BRAZIL',
 'BULGARIA',
 'BURKINA FASO',
 'BURUNDI',
 'CAMBODIA',
 'CAMEROON',
 'CANADA',
 'CENTRAL AFRICAN REPUBLIC',
 'CHAD',
 'COMOROS',
 'DEMOCRATIC REPUBLIC OF THE CONGO',
 "COTE DIVOIRE",
 'CROATIA',
 'CUBA',
 'CYPRUS',
 'CZECH REPUBLIC',
 'DENMARK',
 'EL SALVADOR',
 'ESTONIA',
 'FINLAND',
 'FRANCE',
 'GABON',
 'GEORGIA',
 'GERMANY',
 'GHANA',
 'GREECE',
 'GRENADA',
 'GUATEMALA',
 'GUINEA',
 'GUYANA',
 'HAITI',
 'HONDURAS',
 'HUNGARY',
 'ICELAND',
 'IRAQ',
 'IRELAND',
 'ISRAEL',
 'ITALY',
 'JAPAN',
 'JORDAN',
 'KAZAKHSTAN',
 'KOREA',
 'KOSOVO',
 'KYRGYZSTAN',
 'LATVIA',
 'LIBERIA',
 'LITHUANIA',
 'LUXEMBOURG',
 'MACEDONIA',
 'MADAGASCAR',
 'MALI',
 'MALTA',
 'MOLDOVA',
 'MONTENEGRO',
 'MOZAMBIQUE',
 'NEPAL',
 'NETHERLANDS',
 'NICARAGUA',
 'NIGER',
 'NIGERIA',
 'NORWAY',
 'PANAMA',
 'PAPUA NEW GUINEA',
 'PHILIPPINES',
 'POLAND',
 'PORTUGAL',
 'ROMANIA',
 'RUSSIAN FEDERATION',
 'RWANDA',
 'SAINT KITTS AND NEVIS',
 'SAINT LUCIA',
 'SAMOA',
 'SENEGAL',
 'SERBIA',
 'SEYCHELLES',
 'SINGAPORE',
 'SLOVAKIA',
 'SLOVENIA',
 'SOLOMON ISLANDS',
 'SOUTH AFRICA',
 'SPAIN',
 'SRI LANKA',
 'SUDAN',
 'SURINAME',
 'SWAZILAND',
 'SWEDEN',
 'SWITZERLAND',
 'TAJIKISTAN',
 'TIMOR-LESTE',
 'TONGA',
 'TRINIDAD AND TOBAGO',
 'TURKEY',
 'TURKMENISTAN',
 'UGANDA',
 'UKRAINE',
 'UNITED KINGDOM',
 'UZBEKISTAN']

In [11]:
unicor_uei = tuple(unicor_uei)
Ed_PIID = tuple(Ed_PIID)
excluded_countries = tuple(excluded_countries)

In [12]:
%%sql
not_eligible <<
SELECT recipient_uei, foreign_funding, awarding_office_code, parent_award_id_piid, award_id_piid,
    funding_agency_code, contingency_humanitarian_or_peacekeeping_operation_code, primary_place_of_performance_country_name,
    dod_acquisition_program_code, federal_action_obligation
       from read_csv_auto('{{filepath}}')
       where (disaster_emergency_fund_codes_for_overall_award like '1%' or disaster_emergency_fund_codes_for_overall_award like 'Z%') and (
recipient_uei in {{unicor_uei}} or
           foreign_funding in ('A', 'B') or
           awarding_office_code in ('H94002', 'HT9402', '00NAF') or
           (parent_award_id_piid in {{Ed_PIID}}) or (award_id_piid in {{Ed_PIID}}) or
       (funding_agency_code = '097' AND (
          contingency_humanitarian_or_peacekeeping_operation_code = 'A'
          OR 
          (contingency_humanitarian_or_peacekeeping_operation_code = 'B' AND primary_place_of_performance_country_name != 'UNITED STATES')
          OR
          (contingency_humanitarian_or_peacekeeping_operation_code = 'D' AND primary_place_of_performance_country_name != 'UNITED STATES')
          OR
          (
            dod_acquisition_program_code = 'ZSF' AND 
            primary_place_of_performance_country_name != 'UNITED STATES'
          )
          OR primary_place_of_performance_country_name IN {{excluded_countries}}
           -- doesn't include DoD exclusion for foreign-award, foreign-performed contracts because the place of award is not available in USASpending.gov
        )
        )
           )
order by foreign_funding

In [13]:
not_eligible_total = not_eligible.sum(numeric_only=True)

In [17]:
not_eligible_total

federal_action_obligation   184,380,601
dtype: float64

In [14]:
print(BIL_outlays)

  Minority_Representation          total  small_business           SDB  \
0                     NaN 64,146,985,335   5,811,362,474 2,956,568,462   
1                    None 61,769,560,995   3,562,108,616   913,439,851   
2         Native_Am_Owned    854,075,252     849,693,498   825,809,457   
3       Hispanic_Am_Owned    444,303,486     438,701,687   377,676,745   
4    South_Asian_Am_Owned    432,217,507     346,542,684   321,944,957   
5      Asian_Pac_Am_Owned    336,881,261     327,245,303   242,053,652   
6          Black_Am_Owned    284,856,012     267,958,430   257,524,851   
7    Other_Minority_Owned     25,090,824      19,112,256    18,118,950   

           WOSB        VOSB  
0 1,028,874,567 515,369,097  
1   448,298,811 324,335,962  
2    55,100,071  19,714,715  
3   116,592,470  44,320,804  
4   145,284,551   6,247,774  
5   157,151,777  13,371,359  
6    97,140,709 106,561,974  
7     9,306,179     816,509  
